In [2]:
#정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘인 '앙상블 학습 알고리즘'을 알아보자!

In [6]:
#랜덤 포레스트 - 앙상블 학습의 대표 주자 중 하나
#각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 한 샘플이 중복되어 추출될 수도 있다. -> 부트스트랩 샘플

#랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합 되는 것을 막아준다.

In [8]:
#판다스로 데이터셋 불러온 후, 훈련 세트와 테스트 세트로 나누기

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [10]:
#랜덤 포레스트 클래시파이어 클래스를 화이트 와인 분류 문제에 적용
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state=42) #n_jobs = -1 -> 모든 cpu 코어를 사용

scores = cross_validate(rf, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #test_score은 검증 폴드의 점수 임을 기억하자!

0.9973541965122431 0.8903229806766861


In [12]:
#랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다.
#특히, 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산하는 것 또한 제공 -> 각 결정 트리의 특성 중요도를 취합한다.

rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


In [14]:
#두 번째 특성인 당도의 중요성이 감소하고, 알코올 도수와 pH 특성의 중요도가 조금 상승했다.
#이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다. -> 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.

In [16]:
#부트스트랩 샘플에 포함되지 않고 남는 샘플을 OOB 샘플이라고 한다.
#이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. (마치 검증 세트의 역할!)

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8945545507023283


In [18]:
#교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻었다.
#OOB 점수를 사용하면 교차 검증을 대신할 수 있어 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

In [20]:
#다음으로 알아볼 앙상블 학습 - 랜덤 포레스트와 아주 비슷한 엑스트라 트리!
#랜덤 포레스트와 엑스트라 트리의 차이점 : 부트스트랩 샘플을 사용하지 않는다. -> 전체 훈련 세트를 사용, 대신 노드를 분할할 때 무작위로 분할!
#하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트의 점수를 높인다.

In [22]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [24]:
#랜덤 포레스트와 비슷한 결과를 얻었다.
#이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않다.

#보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다.
#하지만 랜덤하게 노드를 분할하 때문에 빠른 계산 속도가 엑스트라 트리의 장점이다.

In [26]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [28]:
#앙상블 학습 - 그레이디언트 부스팅

#깊이가 얕은 결정 트리 사용하여 이전 트리의 오차를 보완하는 방식
#깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.
#경사 하강법 사용 - 결정 트리를 계속 추가하면서 손실 함수가 가장 낮은 곳을 찾아 이동한다.

In [30]:
#교차 검증 점수 확인

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [32]:
#거의 과대적합이 되지 않는다!!

#결정 트리의 개수를 늘려보자.
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42) #결정 트리의 개수 500개, 학습률 0.2 (기본값은 0.1)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [34]:
#결정 트리 개수를 늘려도 과대적합을 잘 억제하고 있음을 알 수 있다.
#특성 중요도 확인

gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15881044 0.67988912 0.16130044]


In [36]:
#트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample이라는 매개변수가 하나 있다.
#이 매개변수의 기본값은 1.0으로, 전체 훈련 세트를 사용한다. 하지만 1.0보다 작으면 훈련 세트의 일부를 사용한다.
#이는 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷하다.

In [38]:
#히스토그램 기반 그레이디언트 부스팅

#입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.
#256개의 구간 중에 하나를 떼어 놓고 누락된 값을 할당하는 것으로 사용한다. -> 전처리 필요 x

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [42]:
#과대적합을 잘 억제하면서 그레이디언트 부스팅보다 좀 더 높은 성능을 제공한다.

#특성 중요도 계산 -> permutation_importance() 함수 사용 / 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 어떤 특성이 중요한지 계산

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1) #n_repeats는 랜덤하게 섞을 횟수 지정
print(result.importances_mean) #함수가 반환하는 객체는 반복하여 얻은 특성 중요도, 평균, 표준편차를 담고 있다.

[0.08876275 0.23438522 0.08027708]


In [44]:
#테스트 세트에서 특성 중요도 계산하기

result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [46]:
#테스트 세트에서의 성능을 최종적으로 확인

hgb.score(test_input, test_target)

0.8723076923076923

In [50]:
#XGBoost 라이브러리도 그레디언트 부스팅 알고리즘을 구현하였다. 그 외에도 다양한 부스팅 알고리즘을 지원한다.
!pip install xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42) #히스토그램 기반 그레디언트 부스팅 알고리즘 사용
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.7 MB/s eta 0:00:00a 0:00:01
0.9567059184812372 0.8783915747390243


In [54]:
#LightGBM 라이브러리 사용
!pip install lightgbm
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.5 MB/s eta 0:00:00a 0:00:01
0.935828414851749 0.8801251203079884
